In [1]:
import lasagne
import theano
import theano.tensor as T
import numpy as np
from scipy.io import loadmat
import matplotlib.pylab as plt
%matplotlib inline
from IPython import display
import time

import numpy as np
import theano
import theano.tensor as T
from lasagne import nonlinearities
from lasagne import init
from lasagne.utils import unroll_scan

from lasagne.layers import MergeLayer, Layer
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import helper

WARNING (theano.configdefaults): Only clang++ is supported. With g++, we end up with strange g++/OSX bugs.


Make sure we generate the same data every time by fixing the seed

In [2]:
np.random.seed(42)

Data is the (x,y) position of a ball as it ricochets around in a box of size (xmin,xmax,ymin,ymax)

In [27]:
xmin,xmax,ymin,ymax = (-1,1,-1,1)
lower_bound = np.array([xmin,ymin])
upper_bound = np.array([xmax,ymax])

ndims = 2
nsteps = 40
nsequences = 40

def bounce_ball(pos,vel):
    for n in range(1,nsteps):
        pos_next = pos[n-1,:] + vel
        within_boundary = [lower_bound[d] < pos_next[d] < upper_bound[d] for d in range(ndims)]
        if not all(within_boundary):
            vel[~np.array(within_boundary)] *= -1
        pos[n,:] = pos[n-1,:]+vel
    return pos


x = np.zeros((nsequences,nsteps,1,2))
for s in range(nsequences):
    x0 = np.random.rand()*(xmax-xmin)+xmin
    y0 = np.random.rand()*(ymax-ymin)+ymin
    x[s,0,0,:] = np.array([x0,y0])

    vel = np.random.rand(2)
    vel = vel/(10*np.linalg.norm(vel))
    x[s,:,0,:] = bounce_ball(x[s,:,0,:],vel)

In [29]:
#plt.plot(*x[5,:,:].T,'.')
#hhh = T.zeros([3,4,5,6])
#print(hhh.shape.eval()[1])
xTest = x #np.reshape(x[:,1:,:,:],(nsequences,nsteps-1,2,ndims))
xTest.shape
#yTest = np.reshape(x[:,0:(nsteps-1),:,:],(nsequences,(nsteps-1),2,ndims))
#yTest.shape

(40, 40, 1, 2)

In [5]:
#pos = x[3]
#for n in range(nsteps):
#    display.display(plt.clf());
#    #plt.plot(*pos[n,:].T,'.',color='b');
#    plt.xlim(-1,1);
#    plt.ylim(-1,1);
#    display.clear_output(wait=True);
#    display.display(plt.gcf());

Lasagne's GRUlayer expects a three-dimensional input (batch_size,seq_len,dimension_of_signal)

In [6]:
X = x

In [7]:
X.shape

(40, 40, 2, 2)

The output is supposed to be the next frame

In [8]:
#x = np.zeros([5,9,100,50])
#y = np.zeros([5,9,100,50])

In [9]:
#x.shape,y.shape

Lasagne also requires float32's for some reason (probably to save space)

In [10]:
xTest = xTest.astype(np.float32)
yTest = yTest.astype(np.float32)

Since Lasagne won't work with singular hidden units, the model has two, but we only use one in the cost function.

In [11]:
NUM_INPUTS = 1
NUM_UNITS_ENC = 30

In [12]:
#symbolic theano variables. Note that we are using imatrix for X since it goes into the embedding layer
x_sym = T.matrix()

In [13]:
x_sym = T.ftensor4()
y_sym = T.ftensor4()

In [14]:
#
#imported example to understand theno scan
#
import numpy as np
import theano
import theano.tensor as T

up_to = T.iscalar("up_to")

# define a named function, rather than using lambda
def accumulate_by_adding(arange_val, sum_to_date):
    return sum_to_date + arange_val
    #return arange_val
seq = T.arange(up_to)

# An unauthorized implicit downcast from the dtype of 'seq', to that of
# 'T.as_tensor_variable(0)' which is of dtype 'int8' by default would occur
# if this instruction were to be used instead of the next one:
# outputs_info = T.as_tensor_variable(0)

outputs_info = T.as_tensor_variable(np.asarray(0, seq.dtype))
scan_result, scan_updates = theano.scan(fn=accumulate_by_adding,
                                        outputs_info=outputs_info,
                                        sequences=seq)
triangular_sequence = theano.function(inputs=[up_to], outputs=scan_result)

# test
some_num = 15
print(triangular_sequence(some_num))
print([n * (n + 1) // 2 for n in range(some_num)])

[  0   1   3   6  10  15  21  28  36  45  55  66  78  91 105]
[0, 1, 3, 6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91, 105]


First we try with a batch size of one, to test the code with long waiting times

In [15]:
#x_first = x[0:10,:].astype(np.float32)
#y_first = np.zeros([10,99,4]).astype(np.float32);
#y_first[0:10,:,0:1] = y[0:10,:,0:1].astype(np.float32);
#y_first = y[0:10,:,0].astype(np.float32)

#x_first = x[:,:].astype(np.float32)
#y_first = np.zeros([10,99,4]).astype(np.float32);
#y_first[:,:,0:1] = y[:,:,0:1].astype(np.float32);

#y = y_first;
#x = x_first;

In [16]:
#x_first.shape,y_first.shape
class DummyLayer(lasagne.layers.Layer):
    def __init__(self, incoming, shape, **kwargs):
        super(DummyLayer, self).__init__(incoming, **kwargs)
        self._shape = shape
    def get_output_for(self, input, **kwargs):
        #stupid workaround due to stupid python code-checking... Wish this was C++...
        return input.sum()*T.zeros(self._shape)
    def get_output_shape_for(self, input_shape):
        return (self._shape)

In [17]:
class OurLayer(lasagne.layers.CustomRecurrentLayer):
# Create single recurrent computation step function
    def __init__(self, incoming,
            nonlinearity=lasagne.nonlinearities.rectify,
            hid_init=lasagne.init.Constant(0.),
            backwards=False,
            learn_init=False,
            gradient_steps=-1,
            grad_clipping=0,
            unroll_scan=False,
            precompute_input=True,
            mask_input=None,
            only_return_final=False,
            num_layers=2,
            name=None,
            **kwargs):
        if isinstance(incoming, tuple):
            input_shape = incoming
        else:
            input_shape = incoming.output_shape
        
        #l_in_to_hid = lasagne.layers.DenseLayer(lasagne.layers.InputLayer((None,) + input_shape[2:]),num_units=num_layers)
        #l_hid_to_hid = lasagne.layers.DenseLayer(lasagne.layers.InputLayer(([None], num_layers)),num_units=num_layers)
        
        l_in_to_hid  = DummyLayer(lasagne.layers.InputLayer( ((input_shape[0]*input_shape[1]),input_shape[2:]              ) ),shape=((input_shape[0]*input_shape[1]), (input_shape[2]), (input_shape[3]), 3, (num_layers)))
        l_hid_to_hid = DummyLayer(lasagne.layers.InputLayer( ((input_shape[0]*input_shape[1]), (input_shape[2]), (input_shape[3]), 3, (num_layers)) ),shape=((input_shape[0]*input_shape[1]), (input_shape[2]), (input_shape[3]), 3, (num_layers)))
        
        #make sure our parent works correctly:
        lasagne.layers.CustomRecurrentLayer.__init__(self,incoming,input_to_hidden=l_in_to_hid,hidden_to_hidden=l_hid_to_hid,name=name)
        
        #l_test = lasagne.layers.ReshapeLayer(lasagne.layers.InputLayer((None,) + input_shape[2:]),shape=[100,50,5,9])
        
        #print(l_hid_to_hid.input_shape)
        #print(l_hid_to_hid.output_shape)
        
        #print(l_in_to_hid.input_shape)
        #print(l_in_to_hid.output_shape)
        
        self.num_batch = input_shape[0]
        
        self.WE = []
        self.WR = []
        self.WRR = []
        self.WA = []
        for j in range(0,num_layers):
            name='wE'+'_layer_'+str(j)
            self.WE.append( self.add_param(lasagne.init.Normal(0.01),shape=((input_shape[2]*input_shape[3],input_shape[2]*input_shape[3])),name=name) )
            name='wR'+'_layer_'+str(j)
            self.WR.append( self.add_param(lasagne.init.Normal(0.01),shape=((input_shape[2]*input_shape[3],input_shape[2]*input_shape[3])),name=name) )
            if (j == num_layers-1):
                self.WRR.append(1)
            else:
                name='wRR'+'_layer_'+str(j)
                self.WRR.append( self.add_param(lasagne.init.Normal(0.01),shape=((input_shape[2]*input_shape[3],input_shape[2]*input_shape[3])),name=name) )
            name='wA'+'_layer_'+str(j)
            self.WA.append( self.add_param(lasagne.init.Normal(0.01),shape=((input_shape[2]*input_shape[3],input_shape[2]*input_shape[3])),name=name) )
        
        #print(l_op1.input_shape)
        #print(l_op1.output_shape)
        
        #print( (None,) + input_shape[2:] )
        
        #initialize our own data:
        self.num_layers=num_layers
        self.input_shape = input_shape
        #figure out where to init hid_previous, hid_now
        self.has_inited_step = 0
        
    
    def get_output_for(self, inputs, **kwargs):
        """
        Compute this layer's output function given a symbolic input variable.
        Parameters
        ----------
        inputs : list of theano.TensorType
            `inputs[0]` should always be the symbolic input variable.  When
            this layer has a mask input (i.e. was instantiated with
            `mask_input != None`, indicating that the lengths of sequences in
            each batch vary), `inputs` should have length 2, where `inputs[1]`
            is the `mask`.  The `mask` should be supplied as a Theano variable
            denoting whether each time step in each sequence in the batch is
            part of the sequence or not.  `mask` should be a matrix of shape
            ``(n_batch, n_time_steps)`` where ``mask[i, j] = 1`` when ``j <=
            (length of sequence i)`` and ``mask[i, j] = 0`` when ``j > (length
            of sequence i)``. When the hidden state of this layer is to be
            pre-filled (i.e. was set to a :class:`Layer` instance) `inputs`
            should have length at least 2, and `inputs[-1]` is the hidden state
            to prefill with.
        Returns
        -------
        layer_output : theano.TensorType
            Symbolic output variable.
        """
        # Retrieve the layer input
        input = inputs[0]
        # Retrieve the mask when it is supplied
        mask = None
        hid_init = None
        if self.mask_incoming_index > 0:
            mask = inputs[self.mask_incoming_index]
        if self.hid_init_incoming_index > 0:
            hid_init = inputs[self.hid_init_incoming_index]

        # Input should be provided as (n_batch, n_time_steps, n_features)
        # but scan requires the iterable dimension to be first
        # So, we need to dimshuffle to (n_time_steps, n_batch, n_features)
        input = input.dimshuffle(1, 0, *range(2, input.ndim))
        seq_len, num_batch = input.shape[0], input.shape[1]

        #do not precompute!
        self.precompute_input=False
        
        if self.precompute_input:
            # Because the input is given for all time steps, we can precompute
            # the inputs to hidden before scanning. First we need to reshape
            # from (seq_len, batch_size, trailing dimensions...) to
            # (seq_len*batch_size, trailing dimensions...)
            # This strange use of a generator in a tuple was because
            # input.shape[2:] was raising a Theano error
            trailing_dims = tuple(input.shape[n] for n in range(2, input.ndim))
            input = T.reshape(input, (seq_len*num_batch,) + trailing_dims)
            input = helper.get_output(
                self.input_to_hidden, input, **kwargs)

            # Reshape back to (seq_len, batch_size, trailing dimensions...)
            trailing_dims = tuple(input.shape[n] for n in range(1, input.ndim))
            input = T.reshape(input, (seq_len, num_batch) + trailing_dims)

        # We will always pass the hidden-to-hidden layer params to step
        non_seqs = helper.get_all_params(self.hidden_to_hidden)
        # When we are not precomputing the input, we also need to pass the
        # input-to-hidden parameters to step
        if not self.precompute_input:
            non_seqs += helper.get_all_params(self.input_to_hidden)

        # Create single recurrent computation step function
        def step(input_n, hid_previous, *args):
            #assume input_n: array[N][M]
            #assume hid_previous: [N][M][3][num_layers]

            #print('hello world')
            #print(input_n.shape)
            def eval_fun_bot(hid_now):
                return hid_now
    
            def eval_fun(hid_now):
                return hid_now

            hid_now = hid_previous;
            #hid_now[:,:,:] = 2*hid_previous[:,:,:]
            
            #print(input_n.shape.eval())
            #print(hid_previous.shape)
            
            #assume hid_previous[:][:][0][i]: R on i'th layers.
            #assume hid_previous[:][:][1][i]: E on i'th layers.
            #assume hid_previous[:][:][2][i]: A on i'th layers.
            ##assume hid_previous[:][:][3][1]: Output image from previous state.

            #N = hhh.shape.eval()[0]
            #M = hhh.shape.eval()[1]

            #if (self.has_inited_step == 0):
            #    self.has_inited_step = 1
#            hid_previous = T.zeros([M,N,3,self.num_layers])
#
#            hid_now = T.zeros([M,N,3,self.num_layers])

            #testing/twillight zone:
            #hid_now = T.set_subtensor( (hid_now[5,99,49,2,self.num_layers-1]), 1)
            #input = input_n
            #input = T.set_subtensor( (input_n[:,1000,100]), 1)
        
#
#            #perform top to bottom operations:
            for i in range(self.num_layers-1,-1,-1):
                #add data from layers:
                if (i == (self.num_layers-1)):
                    for j in range(0,self.num_batch):
                        t1 = T.reshape( T.dot(self.WE[i],T.reshape(hid_previous[j,:,:,1,i],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )
                        t2 = T.reshape( T.dot(self.WR[i],T.reshape(hid_previous[j,:,:,0,i],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )
                        hid_now = T.set_subtensor( hid_now[j,:,:,0,i] ,  t1+t2 )
                else:
                    for j in range(0,self.num_batch):
                        t1 = T.reshape( T.dot(self.WE[i],T.reshape(hid_previous[j,:,:,1,i],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )
                        t2 = T.reshape( T.dot(self.WR[i],T.reshape(hid_previous[j,:,:,0,i],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )
                        t3 = T.reshape( T.dot(self.WRR[i],T.reshape(hid_previous[j,:,:,0,i+1],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )   
                        hid_now = T.set_subtensor( hid_now[j,:,:,0,i] ,  t1+t2+t3 )
                
                #    hid_now = T.set_subtensor( (hid_now[:,:,:,0,i]) , hid_previous[:,:,:,0,i] + hid_previous[:,:,:,1,i] + hid_now[:,:,:,0,i+1])

            #perform bottom to top operations:
            for i in range(0,self.num_layers):
                #perform approximations.
                if (i==0):
                    for j in range(0,self.num_batch):
                        #calculate A
                        t1 = T.reshape( T.dot(self.WA[i],T.reshape(hid_previous[j,:,:,0,i],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )
                        hid_now = T.set_subtensor(hid_now[:,:,:,2,i], t1 )
                        #calculate errors:
                        T.set_subtensor(hid_now[:,:,:,1,i], hid_now[:,:,:,2,i] - input_n[:,:,:])
                else:
                    for j in range(0,self.num_batch):
                        t1 = T.reshape( T.dot(self.WA[i],T.reshape(hid_previous[j,:,:,0,i],((self.input_shape[2]*self.input_shape[3],1)) )), ((self.input_shape[2],self.input_shape[3])) )
                        hid_now = T.set_subtensor(hid_now[:,:,:,2,i], t1 )
                        #calculate errors:
                        hid_now = T.set_subtensor(hid_now[:,:,:,1,i], hid_now[:,:,:,2,i] - hid_now[:,:,:,2,i-1])

            return hid_now

        def step_masked(input_n, mask_n, hid_previous, *args):
            # Skip over any input with mask 0 by copying the previous
            # hidden state; proceed normally for any input with mask 1.
            hid = step(input_n, hid_previous, *args)
            hid_out = T.switch(mask_n, hid, hid_previous)
            return [hid_out]

        if mask is not None:
            mask = mask.dimshuffle(1, 0, 'x')
            sequences = [input, mask]
            step_fun = step_masked
        else:
            sequences = input
            step_fun = step

        if not isinstance(self.hid_init, Layer):
            # The code below simply repeats self.hid_init num_batch times in
            # its first dimension.  Turns out using a dot product and a
            # dimshuffle is faster than T.repeat.
            dot_dims = (list(range(1, self.hid_init.ndim - 1)) +
                        [0, self.hid_init.ndim - 1])
            hid_init = T.dot(T.ones((num_batch, 1)),
                             self.hid_init.dimshuffle(dot_dims))

        if self.unroll_scan:
            # Retrieve the dimensionality of the incoming layer
            input_shape = self.input_shapes[0]
            # Explicitly unroll the recurrence instead of using scan
            hid_out = unroll_scan(
                fn=step_fun,
                sequences=sequences,
                outputs_info=[hid_init],
                go_backwards=self.backwards,
                non_sequences=non_seqs,
                n_steps=input_shape[1])[0]
        else:
            # Scan op iterates over first dimension of input and repeatedly
            # applies the step function
            hid_out = theano.scan(
                fn=step_fun,
                sequences=sequences,
                go_backwards=self.backwards,
                outputs_info=[hid_init],
                non_sequences=non_seqs,
                truncate_gradient=self.gradient_steps,
                strict=False)[0]

        # When it is requested that we only return the final sequence step,
        # we need to slice it out immediately after scan is applied
        if self.only_return_final:
            hid_out = hid_out[-1]
        else:
            # dimshuffle back to (n_batch, n_time_steps, n_features))
            hid_out = hid_out.dimshuffle(1, 0, *range(2, hid_out.ndim))

            # if scan is backward reverse the output
            if self.backwards:
                hid_out = hid_out[:, ::-1]

        return hid_out
    

In [ ]:
l_in = lasagne.layers.InputLayer((nsequences,nsteps,1,2), input_var = x_sym)

l_custom = OurLayer(l_in,num_layers=1, name='l_custom')

#l_in2 = lasagne.layers.InputLayer((5,9,100,50), input_var = x_sym)
#l_dummy = DummyLayer(l_in2,shape=(5,9,100,50))

batch_size = 5,
seq_len = 9




#total error:
#l_out_total = lasagne.layers.ConcatLayer([l_out0],axis=3);
#l_out_total = lasagne.layers.ConcatLayer([l_out, l_out],axis=2);

# make sure output layer's range matches the range of input, fx with linear output layer
print(lasagne.layers.get_output(l_custom, inputs={l_in: x_sym}).eval({x_sym: xTest}).shape)
#print(lasagne.layers.get_output(l_dummy, inputs={l_in: x_sym}).eval({x_sym: x}).shape)

In [19]:
print(lasagne.layers.get_all_params(l_custom))

[l_custom.hid_init, l_custom.wE_layer_0, l_custom.wR_layer_0, l_custom.wA_layer_0]


In [20]:
# cost function is mean absolute difference between predicted and actual next step.
predicted_values = lasagne.layers.get_output(l_custom)

cost = T.mean(abs(predicted_values[:,:,:,:,1,0]))

In [21]:
cost.eval({x_sym: xTest})

array(0.0)

In [22]:
all_parameters = lasagne.layers.get_all_params([l_custom], trainable=True)
#all_parameters = all_parameters[0:2]
#print(all_parameters)

#add grad clipping to avoid exploding gradients
all_grads = [T.clip(g,-3,3) for g in T.grad(cost, all_parameters)]
#updates = lasagne.updates.adam(all_grads, all_parameters, learning_rate=0.001)
updates = lasagne.updates.adam(all_grads, all_parameters, learning_rate=0.005)

In [23]:
print("Trainable Model Parameters")
print("-"*40)
for param in all_parameters:
    print(param, param.get_value().shape)
print("-"*40)

Trainable Model Parameters
----------------------------------------
(l_custom.wE_layer_0, (2, 2))
(l_custom.wR_layer_0, (2, 2))
(l_custom.wA_layer_0, (2, 2))
----------------------------------------


In [24]:
# compile training function (in C or somesuch)
train_func = theano.function([x_sym], [cost, predicted_values], updates=updates,mode='FAST_COMPILE') #y_sym

TypeError: ('The following error happened while compiling the node', for{cpu,grad_of_scan_fn}(Elemwise{sub,no_inplace}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{:int64:}.0, Subtensor{::int64}.0, Alloc.0, Alloc.0, Alloc.0, Subtensor{int64}.0, l_custom.wE_layer_0, l_custom.wR_layer_0, l_custom.wA_layer_0), '\n', 'Non-unit value on shape on a broadcastable dimension.', (0, 0, 0, 0, 0), (False, True, False, False, True), 'Container name "None"')

In [ ]:
c = 0
csts = []
preds = []
num_epochs = 300
#pred = np.zeros(y.shape).astype(np.float32)
while c < num_epochs:
    cst, pred = train_func(x,y)
    pred = pred.astype(np.float32)
    csts.append(cst)
    preds.append(pred)
    c += 1
    #print(c)

In [ ]:
#while c < num_epochs*2:
#    cst, pred = train_func(x,pred[:,:,2,y)
#    csts.append(cst)
#    preds.append(pred)
#    c += 1
#    #print(c)

In [ ]:
plt.plot(csts);

In [ ]:
preds[-1][:,98,:]

In [ ]:
print(csts[-1])

Compare prediction generated by a randomly initialized network

In [ ]:
preds[0].shape

In [ ]:
plt.plot(y_first[0,:,0].T.flatten(),preds[0][0,:,0].T.flatten(),'*')

to prediction by trained network (ideal is a straight line)

In [ ]:
plt.plot(y_first[0,:,0].T.flatten(),preds[-1][0,:,0].T.flatten(),'*');

A few sample sequence comparisons:

In [ ]:
sequence_idxs = np.random.choice(10,9,replace=False)

In [ ]:
plt.rcParams['figure.figsize'] = (20,20)

In [ ]:
cc = 0
for i in sequence_idxs:
    cc += 1
    plt.subplot(3,3,cc)
    plt.plot(y_first[i,:,0].T)
    plt.plot(preds[-1][i,:,0].T)